In [2]:
import numpy as np
import tensorflow as tf
from glob import glob
from typing import List
from fnmatch import fnmatch
import matplotlib.pyplot as plt
print(tf.__version__)

2.4.1


In [71]:
#root_path = r'D:\sd_dynObj'
file_pattern = '*_grid.npz'
root_path = r'D:\npz_files'
_MAX_LENGTH =25

# function to get files of same pattern
def get_all_bag_paths(root:str,pattern:str)->List[str]:
    list_file_paths=[]
    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch(name, pattern):
                list_file_paths.append((os.path.join(path, name)))

    return sorted(list_file_paths)

In [72]:
all_grid_paths = get_all_bag_paths(root_path,pattern=file_pattern)
all_init_paths = get_all_bag_paths(root_path,pattern="*_init_path.npz")
all_opt_paths = get_all_bag_paths(root_path,pattern="*_opt_path.npz")
all_lr_bnd_paths = get_all_bag_paths(root_path,pattern="*_lr_bnd.npz")
all_odo_paths = get_all_bag_paths(root_path,pattern="*_odo.npz")

In [73]:
all_odo_paths

['D:\\npz_files\\scenario1\\2021-06-10_08-54-06\\2021-06-10_08-54-07_0_nos95_odo.npz',
 'D:\\npz_files\\scenario1\\2021-06-10_08-54-06\\2021-06-10_08-55-07_1_nos92_odo.npz',
 'D:\\npz_files\\scenario1\\2021-06-10_08-58-25\\2021-06-10_08-58-27_0_nos96_odo.npz',
 'D:\\npz_files\\scenario1\\2021-06-10_09-02-12\\2021-06-10_09-02-13_0_nos80_odo.npz',
 'D:\\npz_files\\scenario2\\2021-06-10_10-50-09\\2021-06-10_10-50-10_0_nos113_odo.npz',
 'D:\\npz_files\\scenario2\\2021-06-10_10-50-09\\2021-06-10_10-51-10_1_nos119_odo.npz',
 'D:\\npz_files\\scenario2\\2021-06-10_10-53-22\\2021-06-10_10-53-23_0_nos145_odo.npz',
 'D:\\npz_files\\scenario2\\2021-06-10_10-53-22\\2021-06-10_10-54-23_1_nos57_odo.npz',
 'D:\\npz_files\\scenario2\\2021-06-10_10-55-35\\2021-06-10_10-55-35_0_nos159_odo.npz',
 'D:\\npz_files\\scenario2\\2021-06-10_10-55-35\\2021-06-10_10-56-36_1_nos145_odo.npz',
 'D:\\npz_files\\scenario2\\2021-06-10_10-58-19\\2021-06-10_10-58-20_0_nos176_odo.npz',
 'D:\\npz_files\\scenario2\\2021-06-1

In [74]:
print(len(all_grid_paths),len(all_init_paths),len(all_opt_paths),len(all_lr_bnd_paths),len(all_odo_paths))


74 74 74 74 74


In [75]:
assert len(all_grid_paths)==len(all_init_paths)==len(all_opt_paths)==len(all_lr_bns_paths)==len(all_odo_paths)

In [95]:
def load_npz_data(file_path:str,array_name:str):
    return np.load(file_path)[array_name]

def get_names(path:str):
    scene_name = all_grid_paths[file_idx].split(os.sep)[-3]
    folder_name = all_grid_paths[file_idx].split(os.sep)[-2]
    file_name = all_grid_paths[file_idx].split(os.sep)[-1]
    file_prefix = re.split("_nos",file_name)[0]
    return scene_name,folder_name,file_name,file_prefix

In [98]:
file_idx = 0


# Grid map== grid origin(samples==x==y)== grid resolution
in_grid_map = load_npz_data((all_grid_paths[file_idx]),"grid_data")
in_grid_org_res = load_npz_data((all_grid_paths[file_idx]),"grid_org_res")

#Load paths(initial and optimal) and boundaries(left and right)
in_init_path = load_npz_data((all_init_paths[file_idx]),"init_path")

out_opt_path = load_npz_data((all_opt_paths[file_idx]),"opt_path")

in_left_bnd = load_npz_data((all_lr_bnd_paths[file_idx]),"left_bnd")
in_right_bnd = load_npz_data((all_lr_bnd_paths[file_idx]),"right_bnd")

#Car odometry
in_car_odo = load_npz_data((all_odo_paths[file_idx]),"odo_data")

try:
    assert (
        len(in_grid_map)==len(in_grid_org_res)==len(in_init_path)==len(out_opt_path)==len(in_left_bnd)==len(in_right_bnd)==len(in_car_odo)
    )

except AssertionError:
    print(f"Assertion error, all inputs and outputs are not of equal length")

scene_name,folder_name,file_name,file_prefix = get_names(all_grid_paths[file_idx])

print(f"\n \
Grid map:{np.shape(in_grid_map)}\n \
Grid Orig resol:{np.shape(in_grid_org_res)}\n \
Init path:{np.shape(in_init_path)}\n \
Opt Path:{np.shape(out_opt_path)}\n \
Left bnd:{np.shape(in_left_bnd)}\n \
Right bnd{np.shape(in_right_bnd)}\n \
Car x,y,theta:{np.shape(in_car_odo)}\n")


print(f"{scene_name}\n{folder_name}\n{file_name}\n{file_prefix}")


 Grid map:(95, 1536, 1536)
 Grid Orig resol:(95, 3)
 Init path:(95, 25, 2)
 Opt Path:(95, 25, 2)
 Left bnd:(95, 25, 2)
 Right bnd(95, 25, 2)
 Car x,y,theta:(95, 3)

scenario1
2021-06-10_08-54-06
2021-06-10_08-54-07_0_nos95_grid.npz
2021-06-10_08-54-07_0


In [93]:
import re
scene_name = all_grid_paths[file_idx].split("\\")[-3]
folder_name = all_grid_paths[file_idx].split("\\")[-2]
file_name = all_grid_paths[file_idx].split("\\")[-1]
file_prefix = re.split("_nos",file_name)[0]
print(f"{file_prefix}\n {file_name}")

2021-06-10_08-54-07_0
 2021-06-10_08-54-07_0_nos95_grid.npz
